In [1]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

In [2]:
tf.random.set_random_seed(6969)

In [3]:
import utils
from genotypes import Genotype
# tf.enable_eager_execution()
tf.set_random_seed(6969)
np.random.seed(6969)

In [4]:
# tf.enable_eager_execution()

In [5]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate": 0.025,
    "learning_rate_decay": 0.97,
    "learning_rate_min": 0.0001,
    "num_batches_per_epoch": 2000,
    
    "unrolled": True,
    "epochs": 10,
    "train_batch_size": 8,
    "batch_size": 8,
    "eval_batch_size": 8,
    "save": "EXP",
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [8, 8],
    "save_checkpoints_steps": 100,
    "data": "gs://unet-darts/datasets/train-search",
    "model_dir": 'gs://unet-darts/train-search-ckptsss',
    "max_steps": 10000,
    # NEW
    "steps_per_eval": 2,
    "num_train_examples": 16,
    "num_batches_per_epoch": 2,
    
    "seed": 6969,
    #
    
    "use_tpu": False,
    "use_host_call": False,
    "tpu": 'unet-darts',
#     "zone": 'us-central1-f',
#     "project": "isro-nas"
    "zone": None,
    "project": None
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [6]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn(params):
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = args.crop_size[0], args.crop_size[1]

        # Create a dictionary describing the features.  
        image_feature_description = {
            'train_name': tf.FixedLenFeature([], tf.string),  
            'train_x': tf.FixedLenFeature([], tf.string),
            'train_y': tf.FixedLenFeature([], tf.string),
            'valid_name': tf.FixedLenFeature([], tf.string),  
            'valid_x': tf.FixedLenFeature([], tf.string),
            'valid_y': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            train_x = feature['train_x']
            train_y = feature['train_y']
            valid_x = feature['valid_x']
            valid_y = feature['valid_y']
            train_name = feature['train_name']
            valid_name = feature['valid_name']

            train_x = tf.image.decode_png(train_x, channels=3)
            train_y = tf.image.decode_png(train_y, channels=3)
            valid_x = tf.image.decode_png(valid_x, channels=3)
            valid_y = tf.image.decode_png(valid_y, channels=3)
            
            
            train_x = tf.cast(train_x, tf.float32)
            train_x = tf.image.resize(train_x, (W, H))
            train_y = tf.cast(train_y, tf.float32)
            train_y = tf.image.resize(train_y, (W, H))
            valid_x = tf.cast(valid_x, tf.float32)
            valid_x = tf.image.resize(valid_x, (W, H))
            valid_y = tf.cast(valid_y, tf.float32)
            valid_y = tf.image.resize(valid_y, (W, H))
            
            train_y = train_y[:, :, 0]
            train_y = tf.expand_dims(train_y, axis=-1)
            
            valid_y = valid_y[:, :, 0]
            valid_y = tf.expand_dims(valid_y, axis=-1)
            
            return ((train_x, valid_x), (train_y, valid_y))

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size,  drop_remainder=True)

        return dataset
    
    return _input_fn

In [7]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn(params):
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_train = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)
        x_valid = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_valid = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)

        ds = (x_train, x_valid), (y_train, y_valid)
        dataset = tf.data.Dataset.from_tensor_slices(ds)
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)

        dataset = dataset.repeat(num_epochs).batch(batch_size, drop_remainder=True)
        return dataset
    
    return _input_fn

### Model function

In [8]:
class GeneSaver(tf.estimator.SessionRunHook):
    def __init__(self, model, alphas_normal, alphas_reduce):
        self.model = model
        self.alphas_normal = tf.nn.softmax(alphas_normal)
        self.alphas_reduce = tf.nn.softmax(alphas_reduce)
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
        
    def end(self, session):
        alphas_normal, alphas_reduce = session.run([self.alphas_normal, self.alphas_reduce])
        alphas_normal = alphas_normal
        alphas_reduce = alphas_reduce
        
        genotype = self.model.genotype(alphas_normal, alphas_reduce)        
        self.global_step = session.run(self.global_step)
        
        filename = 'final_genotype.{}'.format((self.global_step))
        tf.logging.info("Saving Genotype for step: {}".format(str(self.global_step)))
        utils.write_genotype(genotype, filename)

In [27]:
def one_hot_encode(x, num_classes):
    b, w, h, c = x.shape
    y = tf.reshape(tf.cast(x, tf.int64), (b, w, h))
    y = tf.one_hot(y, num_classes, on_value=1.0, off_value=0.0)
    return y

In [28]:
def model_fn(features, labels, mode):
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)
    
    global_step = tf.train.get_global_step()
    learning_rate_min = tf.constant(args.learning_rate_min)
    
    learning_rate = tf.train.exponential_decay(
        args.learning_rate,
        global_step,
        decay_rate=args.learning_rate_decay,
        decay_steps=args.num_batches_per_epoch,
        staircase=False,
    )
    
    lr = tf.maximum(learning_rate, learning_rate_min)
    
    optimizer = tf.train.MomentumOptimizer(lr, args.momentum)
    criterion = tf.losses.sigmoid_cross_entropy
    
    eval_hooks = None
    
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        (x_train, x_valid) = features
        (y_train, y_valid) = labels

        preds = model(x_train)
        architect = Architect(model, args)
        architect_step = architect.step(input_train=x_train,
                                       target_train=y_train,
                                       input_valid=x_valid,
                                       target_valid=y_valid,
                                       unrolled=args.unrolled,
                                       )


        with tf.control_dependencies([architect_step]):
            w_var = model.get_thetas()
            loss = model._loss(preds, y_train)
            grads = tf.gradients(loss, w_var)
            train_op = optimizer.apply_gradients(zip(grads, w_var), global_step=tf.train.get_global_step())

        
        tf.summary.scalar('learning_rate', lr)
        
    
    elif mode == tf.estimator.ModeKeys.EVAL:
        alphas_normal = model.arch_parameters()[0]
        alphas_reduce = model.arch_parameters()[1]
        gene_saver = GeneSaver(model, alphas_normal, alphas_reduce)
        eval_hooks = [gene_saver]
        
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        preds = model(x_valid)
        loss = model._loss(preds, y_valid)
        
        miou = tf.metrics.mean_iou(
                    labels=one_hot_encode(y_valid, args.num_classes),
                    predictions=one_hot_encode(tf.expand_dims(tf.argmax(preds, axis=-1), axis=-1), args.num_classes),
                    num_classes=args.num_classes
                )
        acc = tf.metrics.accuracy(
                    labels= one_hot_encode(y_valid, args.num_classes),
                    predictions= one_hot_encode(tf.expand_dims(tf.argmax(preds, axis=-1), axis=-1), args.num_classes),
                )
        
        eval_metric_ops = {
            "miou": miou,
            "accuracy": acc
        }
        
    elif mode == tf.estimator.ModeKeys.PREDICT:
        x = features
        y = labels
        preds = model(x)
        prediction_dict = {"predictions": preds}
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = preds)}
    
    # 5. Return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode = mode,
        predictions = prediction_dict,
        loss = loss,
        train_op = train_op,
        eval_metric_ops = eval_metric_ops,
        export_outputs = export_outputs,
        evaluation_hooks=eval_hooks
    )

### Estimator

In [29]:
def get_train():
    return make_inp_fn(filename=tf.gfile.Glob(os.path.join('../../datasets/train_search', 'train-search-*-00008.tfrecords')),
                        mode=tf.estimator.ModeKeys.TRAIN,
                        batch_size=args.batch_size)

In [30]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


In [31]:
config = tf.estimator.RunConfig(save_checkpoints_steps=args.save_checkpoints_steps,
                                model_dir=args.model_dir, tf_random_seed=args.seed)

In [32]:
estimator = tf.estimator.Estimator(model_fn = model_fn, 
                     config=config)
estimator.evaluate(input_fn = get_train(), steps = None)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdf6e2ca810>, '_model_dir': 'gs://unet-darts/train-search-ckptsss', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': 6969, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_timeout_secs': 7200, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}
INFO:tensorflow:Could not find trained model in model_dir: gs://unet-darts

KeyboardInterrupt: 

In [ ]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(output_dir estimator):
    train_spec = tf.estimator.TrainSpec(input_fn = get_train(),
                                    max_steps = 1000)
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = get_train(),
                                  steps = None, throttle_secs=600)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [11]:
train_and_evaluate(args.model_dir, estimator)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbe35be9250>, '_model_dir': './outputdir', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_timeout_secs': 7200, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_experimental_max_worker_delay_secs': None, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation l

## Extra Test

In [57]:
def one_hot_encode(x, num_classes):
    b, w, h, c = x.shape
    y = tf.reshape(tf.cast(x, tf.int64), (b, w, h))
    y = tf.one_hot(y, num_classes, on_value=1.0, off_value=0.0)
    return y

In [106]:
g = tf.compat.v1.Graph()
with g.as_default():
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)
    (x_train, x_valid), (y_train, y_valid) = get_train()(args).make_one_shot_iterator().get_next()
    logits = model(x_train)

    miou, conf_mat = tf.metrics.mean_iou(
        labels=y_train,
        predictions=tf.expand_dims(tf.argmax(logits, axis=-1), axis=-1),
        num_classes=args.num_classes
    )
    global_init_op = tf.global_variables_initializer()
    local_init_op = tf.local_variables_initializer()

In [109]:
g.get_collection(tf.GraphKeys.UPDATE_OPS)

[]

In [107]:
with tf.Session(graph=g) as sess:
    sess.run([global_init_op, local_init_op])
    pred = sess.run(logits)
    conf_mat_out = sess.run(conf_mat)
    miou_out = sess.run(miou)
#     o = sess.run(tf.expand_dims(tf.argmax(logits, axis=-1), axis=-1))
    o = sess.run(o)

FailedPreconditionError: Error while reading resource variable total_confusion_matrix from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/total_confusion_matrix/N10tensorflow3VarE does not exist.
	 [[node Sum/ReadVariableOp (defined at /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for u'Sum/ReadVariableOp':
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  Fi

In [55]:
ds = get_train()(args)
it = ds.make_one_shot_iterator()
(x_train, x_valid), (y_train, y_valid) = it.get_next()
c = 0

In [ ]:
while(50):
    with tf.Session() as sess:
        print(c)
        y = sess.run(y_valid)
        if(np.prod(y >= 6)): 
            print(y)
            break
        c += 1